In [1]:
import torch.nn as nn
import torch.functional as F
import torch
import torch.optim as optim
import math

In [3]:
# Taken from paper
batch_size = 64
hidden_size = 512
embedding_size = 512 #LSTM, Layers
lr = 0.001
skip_num = 0
code_vocab_size = 512
#code_max_len = max_code
comment_vocab_size = 512
#comment_max_len = max_comm # This and max_c

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, code_vocab, comm_vocab):
        #super(ActorNetwork, self).__init__()
        self.hidden_size = hidden_size
        # these are probalby btached, investigate later
        # Create embeeddings
        self.enc_embedding_seq = nn.Embedding(code_vocab, hidden_size)
        self.enc_embedding_struct = nn.Embedding(comm_vocab, hidden_size)

        #encode with LSTM
        self.enc_LSTM_seq = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.enc_LSTM_struct = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        # END ENCODE

    def forward(self, seq, struct):
        '''
        Takes Sequential and Structural representation of code and creates two hidden layers and outputs out of them.
        '''
        e_seq = self.enc_embedding_seq(seq)
        e_struct = self.enc_embedding_struct(struct)
        out_seq, hide_seq = self.enc_LSTM_seq(e_seq)
        out_struct, hide_struct = self.enc_LSTM_struct(e_struct)

        return out_seq, hide_seq, out_struct, hide_struct


class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, code_vocab, comm_vocab):
        # START DCODE
        self.dec_embedding = nn.Embedding(comm_vocab, hidden_size)
        self.relu = nn.ReLU()
        self.dec_LSTM = nn.LSTM(hidden_size, hidden_size)
        self.pred = nn.Linear(hidden_size,  comm_vocab, 1) # Predict 
    def forward(self, input, hidden):
        out = self.dec_embedding(input)
        out = self.relu(out)
        out, hidden = self.dec_LSTM(out, hidden)
        out = self.pred(out)

        return out, hidden 

def Attention(query, key, mask=None, dropout=None): # Key and Value are the? same
    "Compute 'Scaled Dot Product Attention'"

    scores = torch.matmul(query, key.transpose(-2, -1)) 
    attention = F.softmax(scores, dim = -1)
    # Context vector
    return torch.bmm(attention, key), attention

class HybridAttention(nn.Module):
    def __init__(self, hidden_size):
        # START DCODE
        self.linear1 = nn.Linear(hidden_size*2, hidden_size)
        self.linear2 = nn.Linear(hidden_size*2, hidden_size)
        self.tanh = nn.Tanh()
    def forward(self, query, h_str, h_seq):
        # Attention of each hidden state we look at and get context vetcor
        d_str, a_str = Attention(query, h_str)
        d_seq, a_seq = Attention(query, h_seq)
        
        # Get respective context vectors and combine together

        d_t = self.linear1(torch.cat([d_str, d_seq]))

        # Finally create context vector for next wordprediction
        return self.tanh(self.linear2(torch.cat([query, d_t])))
# scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
# #print(scores.shape)
# scores = scores.squeeze(2).unsqueeze(1)
# weights = F.softmax(scores, dim=-1)
# #print(weights.shape, keys.shape)
# context = torch.bmm(weights, keys)

NameError: name 'nn' is not defined

In [ ]:
encoder = Encoder(0, hidden_size, code_vocab_size, comment_vocab_size)
decoder = Decoder(0, hidden_size, code_vocab_size, comment_vocab_size)
encoder_optimizer = optim.Adam(encoder.parameters(), lr = lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr = lr)